In [ ]:
import os
import sys
import importlib.util

sys.path.append('./src/')

from helpers.snakemake import nested_dict_update, get_git_revision_hash
from default_net_params import params as net_params
from data_preprocessing.cytoarchitecture import NeuronNumbers
from data_preprocessing.connectivity import SynapseNumbers
from network import Network

In [ ]:
base_path = os.getcwd()

In [ ]:
# Load script from specified path sys.argv[1] (snakemake {input})
# Does nothing else than `import path/to/script as exp` would do
experiment_file = './experiments/experiment_stabilized_groundstate.py'
conf_name, _ = os.path.splitext(os.path.basename(experiment_file))
conf_path = os.path.join(os.getcwd(), './experiments/experiment_stabilized_groundstate.py')
spec = importlib.util.spec_from_file_location(conf_name, conf_path)
exp = importlib.util.module_from_spec(spec)
spec.loader.exec_module(exp)

In [ ]:
# Update default parameters with parameters specified in exp
nested_dict_update(net_params, exp.net_params)
outpath = net_params['outpath']

In [ ]:
# Create Network class
NN = NeuronNumbers(
    surface_area=net_params['surface_area'],
    **net_params['cytoarchitecture_params']
)

In [ ]:
# Create Synapse class
SN = SynapseNumbers(
    NN=NN,
    **net_params['predictive_connectomic_params']
)

In [ ]:
net = Network(NN, SN, net_params)